### 【 D0127_Mini-Project_이준기 】

- 민원 처리 (NLP)
- 텍스트 -> Task 분류

In [16]:
## 모듈 로딩
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

import pandas as pd

In [17]:
## 데이터 준비
DATA_FILE = '../Project/PrData/train.csv'

df = pd.read_csv(DATA_FILE)

df.nunique()

text     688466
label        18
dtype: int64

In [18]:
## 전처리 함수 생성
from konlpy.tag import Okt
import re

okt = Okt()

stopwords = ['합니다', '바랍니다', '부탁', '요청', '제발', '주세요', '하십시오']

def preprocessing(text):
    text = re.sub('[^가-힣 ]', ' ', text) # 한글 + 공백만 남기기

    nouns = okt.nouns(text)

    nouns = [word for word in nouns if word not in stopwords and len(word) > 1]

    return ' '.join(nouns)

In [19]:
## 전처리 잘 되는지 임시 텍스트로 테스트해보기
# text = "초등학교 바로 근처에 피부샵 간판에 '브라질리언'이라고 써있으니 초등생 아이가 그게 뭐냐고 물어봅니다. 선뜻 대답하기 어려웠고 요즘 아이들은 거의 다 스마트폰이 있으니 검색해 보고 음란 사진에 접할까 봐 염려되니 제재 바랍니다."

# text = preprocessing(text)

# print(text)

In [20]:
## 전처리 적용
df['clean_text'] = df['text'].apply(preprocessing)

In [22]:
X = df['clean_text']
y = df['label']

In [23]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y   # 🔥 라벨 비율 유지
)


In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    max_features=20000,
    ngram_range=(1,2)
)

X_train_vec = vectorizer.fit_transform(X_train)
X_val_vec   = vectorizer.transform(X_val)


In [27]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train_vec, y_train)

print("Validation Accuracy:", model.score(X_val_vec, y_val))

Validation Accuracy: 0.8232
